# Modular License Plate Detection

This notebook demonstrates how to use the modular license plate detection framework. The code has been reorganized into a structured package to improve maintainability, reusability, and extensibility.

In [ ]:
# Setup imports
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow import keras
import sys

# Add project root to path to enable imports
project_root = Path(os.getcwd()).parent
sys.path.insert(0, str(project_root))

# Import our modules
from license_plate_detection.data.loader import get_data_path, load_dataset, preprocess_dataset, split_dataset
from license_plate_detection.data.augmentation import augment_data, visualize_augmentation
from license_plate_detection.models.detector import create_license_plate_detector, create_enhanced_license_plate_detector
from license_plate_detection.models.losses import enhanced_iou_metric, combined_detection_loss, giou_loss
from license_plate_detection.train.trainer import train_model, save_model
from license_plate_detection.train.scheduler import CosineAnnealingWarmRestarts
from license_plate_detection.evaluation.evaluator import evaluate_license_plate_detection, evaluate_model_comprehensive
from license_plate_detection.utils.visualization import visualize_training_history, visualize_model_summary
from license_plate_detection.utils.analysis import analyze_error_patterns
from license_plate_detection.main import detect_license_plate, load_and_prepare_model

print(f"TensorFlow version: {tf.__version__}")
print(f"Project root: {project_root}")

## 1. Dataset Loading and Preprocessing

First, we'll load and preprocess the dataset using our modular functions. The data loading is now handled by dedicated functions that can work with various annotation formats.

In [ ]:
# Get paths to dataset
data_path = get_data_path()
IMAGES_PATH = data_path / "images"
ANNOTATIONS_PATH = data_path / "annotations"

print(f"Data path: {data_path}")
print(f"Images directory: {IMAGES_PATH}")
print(f"Annotations directory: {ANNOTATIONS_PATH}")

# Load the dataset
df = load_dataset(ANNOTATIONS_PATH, IMAGES_PATH)
print(f"Loaded {len(df)} annotated images.")

# Show sample data
df.head()

In [ ]:
# Visualize a sample image from the dataset
if len(df) > 0:
    # Take a specific sample or a random one
    sample_idx = min(42, len(df) - 1)  # Ensure index exists
    sample = df.iloc[sample_idx]
    
    # Load the image
    img = cv2.imread(sample["image_path"])
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    # Draw bounding box
    x, y, w, h = sample["x"], sample["y"], sample["w"], sample["h"]
    cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    # Display
    plt.figure(figsize=(10, 8))
    plt.imshow(img_rgb)
    if "plate_text" in sample and sample["plate_text"] != "Unknown":
        plt.title(f"Plate: {sample['plate_text']}")
    plt.axis('off')
    plt.show()
else:
    print("No images loaded. Please check dataset path.")

## 2. Data Preprocessing

Now we'll preprocess the dataset by resizing images and normalizing bounding box coordinates. This is handled by the `preprocess_dataset` function.

In [ ]:
# Preprocess the dataset
IMAGE_SIZE = (224, 224)  # Target size for CNN
X, y = preprocess_dataset(df, image_size=IMAGE_SIZE)

print(f"Processed {len(X)} images.")
print("Image shape:", X[0].shape)
print("Sample bounding box (normalized):", y[0])

# Visualize a processed sample to see the difference between an original and preprocessed image
def visualize_processed_sample(index=0):
    if index >= len(X) or index < 0:
        print(f"Index {index} is out of bounds.")
        return
        
    img_normalized = X[index]
    bbox_norm = y[index]
    original_row = df.iloc[index]
    
    # Load the original image
    img_original = cv2.imread(original_row["image_path"])
    img_original = cv2.cvtColor(img_original, cv2.COLOR_BGR2RGB)
    
    # Draw original bbox
    x_orig, y_orig, w_orig, h_orig = original_row["x"], original_row["y"], original_row["w"], original_row["h"]
    img_original_vis = img_original.copy()
    cv2.rectangle(img_original_vis, (x_orig, y_orig), (x_orig + w_orig, y_orig + h_orig), (255, 0, 0), 2)
    
    # Prepare normalized image
    img_vis = (img_normalized * 255).astype(np.uint8).copy()
    x_norm = int(bbox_norm[0] * IMAGE_SIZE[0])
    y_norm = int(bbox_norm[1] * IMAGE_SIZE[1])
    w_norm = int(bbox_norm[2] * IMAGE_SIZE[0])
    h_norm = int(bbox_norm[3] * IMAGE_SIZE[1])
    cv2.rectangle(img_vis, (x_norm, y_norm), (x_norm + w_norm, y_norm + h_norm), (0, 255, 0), 2)
    
    # Plot side-by-side
    fig, axs = plt.subplots(1, 2, figsize=(12, 5))
    axs[0].imshow(img_original_vis)
    axs[0].set_title('Original Image with Original BBox')
    axs[0].axis('off')
    
    axs[1].imshow(img_vis)
    axs[1].set_title('Normalized & Resized Image with BBox')
    axs[1].axis('off')
    
    plt.tight_layout()
    plt.show()

# Show a processed sample
visualize_processed_sample(0)

## 3. Data Augmentation

To improve model generalization, we'll use data augmentation techniques. The augmentation module provides functions to apply various transformations to our dataset.

In [ ]:
# Apply data augmentation
X_aug, y_aug = augment_data(X, y, augmentation_factor=1)

# Visualize some augmented samples
visualize_augmentation(X, y, X_aug, y_aug, num_samples=2)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = split_dataset(X_aug, y_aug, test_size=0.2, random_state=42)
print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}")

## 4. Model Architecture

Now we'll create our license plate detection model. We'll use the enhanced model architecture for better performance.

In [ ]:
# Create the enhanced model
model = create_enhanced_license_plate_detector(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

# Display model summary
model.summary()

# Print model size information
trainable_count = np.sum([keras.backend.count_params(w) for w in model.trainable_weights])
non_trainable_count = np.sum([keras.backend.count_params(w) for w in model.non_trainable_weights])
print(f'Total parameters: {trainable_count + non_trainable_count:,}')
print(f'Trainable parameters: {trainable_count:,}')
print(f'Non-trainable parameters: {non_trainable_count:,}')

# Visualize model architecture (optional)
# visualize_model_summary(model)

## 5. Model Training

Now we'll train the model using our custom loss functions and learning rate scheduling.

In [ ]:
# Compile model
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=combined_detection_loss,
    metrics=[enhanced_iou_metric, giou_loss]
)

# Create callbacks
callbacks = [
    # Learning rate scheduler
    CosineAnnealingWarmRestarts(
        initial_lr=0.001,
        min_lr=1e-6,
        cycles=5,
        cycle_length=10
    ),
    
    # Early stopping
    keras.callbacks.EarlyStopping(
        monitor='val_enhanced_iou_metric',
        patience=10,
        restore_best_weights=True,
        mode='max'
    ),
    
    # Model checkpoint
    keras.callbacks.ModelCheckpoint(
        'enhanced_license_plate_detector.h5',
        monitor='val_enhanced_iou_metric',
        save_best_only=True,
        mode='max',
        verbose=1
    )
]

# Train model (commented out to avoid accidental training)
"""
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=16,
    callbacks=callbacks,
    verbose=1
)

# Visualize training history
visualize_training_history(history)

# Save model
save_model(model, 'license_plate_detector_final.h5')
"""

# Alternatively, we can use the train_model function from our module
"""
history, model = train_model(
    model=model,
    X_train=X_train,
    y_train=y_train,
    X_val=X_val,
    y_val=y_val,
    loss_function=combined_detection_loss,
    metrics=[enhanced_iou_metric],
    epochs=50,
    batch_size=16,
    callbacks=callbacks
)
"""

## 6. Model Evaluation

Let's evaluate our model using different evaluation functions. If you don't have a trained model yet,
you can load a pre-trained model or use the following code to generate some sample predictions for demonstration.

In [ ]:
# For demonstration purposes, we'll use the untrained model to generate random predictions
# In a real scenario, you would load a trained model

# Option 1: Load a pre-trained model if available
"""
model = load_and_prepare_model('license_plate_detector.h5')
"""

# Option 2: For demo purposes, use the untrained model with random but plausible predictions
def generate_demo_predictions(model, X_val, y_val):
    """Generate plausible demo predictions for visualization purposes"""
    # Get predictions from untrained model
    random_preds = model.predict(X_val)
    
    # Make them somewhat reasonable by combining with ground truth
    demo_preds = []
    for i, (pred, gt) in enumerate(zip(random_preds, y_val)):
        # Create a prediction that's a noisy version of ground truth
        noise = np.random.normal(0, 0.1, 4)  # Add some noise
        noisy_pred = gt + noise
        
        # Ensure values are in range [0, 1]
        noisy_pred = np.clip(noisy_pred, 0, 1)
        
        # Make width and height reasonable
        noisy_pred[2] = max(0.05, min(noisy_pred[2], 0.5))  # Width
        noisy_pred[3] = max(0.05, min(noisy_pred[3], 0.3))  # Height
        
        demo_preds.append(noisy_pred)
    
    return np.array(demo_preds)

# Generate demo predictions
demo_predictions = generate_demo_predictions(model, X_val, y_val)

# Basic evaluation with visualization
def evaluate_with_demo_predictions(X_val, y_val, y_pred):
    """Wrapper to evaluate with provided predictions"""
    # Modified version of evaluate_license_plate_detection that accepts y_pred directly
    iou_values = []
    confidences_list = []  # For consistency with original function
    
    for i in range(len(y_val)):
        true_bbox = y_val[i]
        pred_bbox = y_pred[i]
        
        # Convert to x1, y1, x2, y2 format
        x1_true, y1_true = true_bbox[0], true_bbox[1]
        x2_true, y2_true = x1_true + true_bbox[2], y1_true + true_bbox[3]

        x1_pred, y1_pred = pred_bbox[0], pred_bbox[1]
        x2_pred, y2_pred = x1_pred + pred_bbox[2], y1_pred + pred_bbox[3]
        
        # Calculate intersection
        x1_inter = max(x1_true, x1_pred)
        y1_inter = max(y1_true, y1_pred)
        x2_inter = min(x2_true, x2_pred)
        y2_inter = min(y2_true, y2_pred)
        
        # Calculate areas
        w_intersect = max(0, x2_inter - x1_inter)
        h_intersect = max(0, y2_inter - y1_inter)
        area_intersect = w_intersect * h_intersect
        
        area_true = true_bbox[2] * true_bbox[3]
        area_pred = pred_bbox[2] * pred_bbox[3]
        area_union = area_true + area_pred - area_intersect
        
        # IoU
        iou = area_intersect / area_union if area_union > 0 else 0
        iou_values.append(iou)
        
        # Dummy confidence value for visualization
        confidences_list.append(0.8 + 0.2 * np.random.random())
    
    # Find best and worst predictions
    num_samples = 4  # Number of samples to visualize
    iou_indices = np.argsort(iou_values)
    worst_indices = iou_indices[:num_samples//2]
    best_indices = iou_indices[-num_samples//2:]
    
    # Visualization
    plt.figure(figsize=(15, 4*num_samples))
    
    samples_to_show = np.concatenate([worst_indices, best_indices])
    
    for i, idx in enumerate(samples_to_show):
        img = X_val[idx]
        true_bbox = y_val[idx]
        pred_bbox = y_pred[idx]
        
        # Display image with both bounding boxes
        img_display = (img * 255).astype(np.uint8).copy()
        h, w = img.shape[:2]
        
        # True bbox (green)
        x, y = int(true_bbox[0] * w), int(true_bbox[1] * h)
        bbox_w, bbox_h = int(true_bbox[2] * w), int(true_bbox[3] * h)
        cv2.rectangle(img_display, (x, y), (x + bbox_w, y + bbox_h), (0, 255, 0), 2)
        
        # Pred bbox (red)
        x, y = int(pred_bbox[0] * w), int(pred_bbox[1] * h)
        bbox_w, bbox_h = int(pred_bbox[2] * w), int(pred_bbox[3] * h)
        cv2.rectangle(img_display, (x, y), (x + bbox_w, y + bbox_h), (255, 0, 0), 2)
        
        # Add confidence text
        confidence = confidences_list[idx]
        cv2.putText(img_display, f"{confidence:.2f}", (x, y-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        
        plt.subplot(num_samples, 2, i+1)
        plt.imshow(img_display)
        plt.title(f"IoU: {iou_values[idx]:.4f} {'(Worst)' if idx in worst_indices else '(Best)'}", fontsize=10)
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print("Overall Performance:")
    print(f"Average IoU: {np.mean(iou_values):.4f}")
    print(f"Median IoU: {np.median(iou_values):.4f}")
    print(f"Min IoU: {np.min(iou_values):.4f}")
    print(f"Max IoU: {np.max(iou_values):.4f}")
    
    return iou_values

# Run evaluation with demo predictions
iou_values = evaluate_with_demo_predictions(X_val, y_val, demo_predictions)

## 7. Error Analysis

Now we'll analyze the error patterns to understand where the model struggles and identify opportunities for improvement.

In [ ]:
# Analyze error patterns with demo predictions
def analyze_error_patterns_with_demo_predictions(X_val, y_val, y_pred, plate_sizes=None):
    """Modified version of analyze_error_patterns that accepts y_pred directly"""
    print("Analyzing error patterns...")
    
    # If plate sizes not provided, calculate them
    if plate_sizes is None:
        plate_sizes = [box[2] * box[3] for box in y_val]
    
    # Calculate errors and IoU
    iou_values = []
    x_errors = []
    y_errors = []
    w_errors = []
    h_errors = []
    center_errors = []
    area_errors = []
    
    for i in range(len(y_val)):
        true_box = y_val[i]
        pred_box = y_pred[i]
        
        # Basic coordinate errors
        x_errors.append(abs(true_box[0] - pred_box[0]))
        y_errors.append(abs(true_box[1] - pred_box[1]))
        w_errors.append(abs(true_box[2] - pred_box[2]))
        h_errors.append(abs(true_box[3] - pred_box[3]))
        
        # Center point error
        true_center_x = true_box[0] + true_box[2]/2
        true_center_y = true_box[1] + true_box[3]/2
        pred_center_x = pred_box[0] + pred_box[2]/2
        pred_center_y = pred_box[1] + pred_box[3]/2
        center_errors.append(np.sqrt((true_center_x - pred_center_x)**2 + 
                                    (true_center_y - pred_center_y)**2))
        
        # Area error
        true_area = true_box[2] * true_box[3]
        pred_area = pred_box[2] * pred_box[3]
        area_errors.append(abs(true_area - pred_area) / true_area)
        
        # Calculate IoU
        # Convert to x1, y1, x2, y2 format
        x1_true, y1_true = true_box[0], true_box[1]
        x2_true, y2_true = x1_true + true_box[2], y1_true + true_box[3]

        x1_pred, y1_pred = pred_box[0], pred_box[1]
        x2_pred, y2_pred = x1_pred + pred_box[2], y1_pred + pred_box[3]
        
        # Calculate intersection
        x1_inter = max(x1_true, x1_pred)
        y1_inter = max(y1_true, y1_pred)
        x2_inter = min(x2_true, x2_pred)
        y2_inter = min(y2_true, y2_pred)
        
        # Calculate areas
        w_intersect = max(0, x2_inter - x1_inter)
        h_intersect = max(0, y2_inter - y1_inter)
        area_intersect = w_intersect * h_intersect
        
        area_true = true_box[2] * true_box[3]
        area_pred = pred_box[2] * pred_box[3]
        area_union = area_true + area_pred - area_intersect
        
        # IoU
        iou = area_intersect / area_union if area_union > 0 else 0
        iou_values.append(iou)
    
    # Characterize plates by size
    small_threshold = 0.03
    large_threshold = 0.1
    
    small_indices = [i for i, size in enumerate(plate_sizes) if size < small_threshold]
    medium_indices = [i for i, size in enumerate(plate_sizes) if small_threshold <= size <= large_threshold]
    large_indices = [i for i, size in enumerate(plate_sizes) if size > large_threshold]
    
    # Calculate error metrics by plate size
    def get_metrics_by_indices(indices, name):
        if not indices:
            print(f"No {name} plates found")
            return None
            
        size_iou = [iou_values[i] for i in indices]
        size_x_err = [x_errors[i] for i in indices]
        size_y_err = [y_errors[i] for i in indices]
        size_w_err = [w_errors[i] for i in indices]
        size_h_err = [h_errors[i] for i in indices]
        size_center_err = [center_errors[i] for i in indices]
        size_area_err = [area_errors[i] for i in indices]
        
        print(f"\n{name} Plates (n={len(indices)}):")
        print(f"  Mean IoU: {np.mean(size_iou):.4f}")
        print(f"  Center Error: {np.mean(size_center_err):.4f}")
        print(f"  Area Error: {np.mean(size_area_err):.4f}")
        print(f"  X Error: {np.mean(size_x_err):.4f}, Y Error: {np.mean(size_y_err):.4f}")
        print(f"  Width Error: {np.mean(size_w_err):.4f}, Height Error: {np.mean(size_h_err):.4f}")
        
        return {
            'mean_iou': np.mean(size_iou),
            'center_error': np.mean(size_center_err),
            'area_error': np.mean(size_area_err),
            'x_error': np.mean(size_x_err),
            'y_error': np.mean(size_y_err),
            'w_error': np.mean(size_w_err),
            'h_error': np.mean(size_h_err)
        }
    
    # Get metrics by plate size
    small_metrics = get_metrics_by_indices(small_indices, "Small")
    medium_metrics = get_metrics_by_indices(medium_indices, "Medium")
    large_metrics = get_metrics_by_indices(large_indices, "Large")
    
    # Create visualizations (can be added based on the original function)
    
    # Return results for potential further use
    return {
        'iou_values': iou_values,
        'x_errors': x_errors,
        'y_errors': y_errors,
        'w_errors': w_errors,
        'h_errors': h_errors,
        'center_errors': center_errors,
        'area_errors': area_errors,
        'small_metrics': small_metrics,
        'medium_metrics': medium_metrics,
        'large_metrics': large_metrics
    }

# Analyze error patterns using the demo predictions
error_analysis = analyze_error_patterns_with_demo_predictions(
    X_val=X_val,
    y_val=y_val,
    y_pred=demo_predictions,
    plate_sizes=[box[2] * box[3] for box in y_val]
)

## 8. License Plate Detection on New Images

Finally, let's use our model to detect license plates in new images. We'll use the `detect_license_plate` function
from our main module.

In [ ]:
# For demonstration purposes, we'll use a sample image from our dataset
if len(df) > 0:
    sample_img_path = df.iloc[10]["image_path"]

    # In a real scenario, you would use a trained model
    # model = load_and_prepare_model('license_plate_detector.h5')

    # For demo purposes, we'll create a function to simulate detection
    def demo_detect_license_plate(image_path):
        """Simulate license plate detection for demonstration"""
        # Load image
        img = cv2.imread(image_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        orig_h, orig_w = img_rgb.shape[:2]
        
        # Get ground truth from dataset if available
        file_name = os.path.basename(image_path)
        matched_rows = df[df["image_path"].str.contains(file_name)]
        
        if len(matched_rows) > 0:
            # Use ground truth with small random offset
            row = matched_rows.iloc[0]
            x, y, w, h = row["x"], row["y"], row["w"], row["h"]
            
            # Add some noise to simulate prediction
            noise_factor = 0.1
            x += int(np.random.normal(0, w * noise_factor))
            y += int(np.random.normal(0, h * noise_factor))
            w += int(np.random.normal(0, w * noise_factor))
            h += int(np.random.normal(0, h * noise_factor))
            
            # Ensure values are valid
            x = max(0, min(x, orig_w - 10))
            y = max(0, min(y, orig_h - 10))
            w = max(10, min(w, orig_w - x))
            h = max(10, min(h, orig_h - y))
        else:
            # Generate random detection
            x = int(orig_w * 0.4)
            y = int(orig_h * 0.4)
            w = int(orig_w * 0.2)
            h = int(orig_h * 0.1)
        
        # Draw detection on image
        result_img = img_rgb.copy()
        cv2.rectangle(result_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Display result
        plt.figure(figsize=(10, 8))
        plt.imshow(result_img)
        plt.axis('off')
        plt.title("License Plate Detection (Demo)")
        plt.show()
        
        # Extract plate region
        plate_region = img_rgb[y:y + h, x:x + w]
        
        # Show extracted plate
        plt.figure(figsize=(6, 2))
        plt.imshow(plate_region)
        plt.axis('off')
        plt.title("Extracted License Plate")
        plt.show()
        
        return plate_region, [x, y, w, h]

    # Detect license plate in sample image
    demo_detect_license_plate(sample_img_path)
else:
    print("No images available for demonstration.")

## 9. Conclusion

This notebook demonstrated how to use the modular license plate detection framework. The modularization offers several benefits:

1. **Code Organization**: Clear separation of concerns with different modules handling specific tasks
2. **Reusability**: Functions can be reused across different projects and contexts
3. **Maintainability**: Easier to maintain and debug code with well-defined interfaces
4. **Extensibility**: New functionality can be added with minimal changes to existing code
5. **Testability**: Functions with clear inputs and outputs are easier to test

For a complete implementation, additional features could be added:

- Implement more advanced augmentation techniques
- Add support for other annotation formats
- Integrate with OCR for license plate text recognition
- Develop real-time detection pipeline using video input